In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
train = pd.read_csv('trainMaster.csv')

# Target

In [3]:
x = train['comment_text']

y1 = train['toxic']
y2 = train['severe_toxic']
y3 = train['obscene']
y4 = train['threat']
y5 = train['insult']
y6 = train['identity_hate']
y7 = train['clean']

datay = [y1, y2, y3, y4, y5, y6, y7]

# Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
dSplit = {}
for j in range(1,8):
        X_train, X_test, dSplit['y{}_train'.format(j)], dSplit['y{}_test'.format(j)] = train_test_split(x, datay[j-1], test_size=0.3, random_state=101)

In [6]:
print('OwO')

OwO


# K Fold

In [7]:
# from sklearn.model_selection import KFold

In [8]:
# K = 5
# kf = KFold(n_splits=K, shuffle=True, random_state = 42)

In [9]:
# dFold = {}
# for i in range(1,8):
#     dFold['Hasilfold{}'.format(i)] = kf.split(x,datay[i-1])

# Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer


# Analyzer for Pipeline

In [11]:
stop = []
for words in stopwords.words('english'):
    s = [char for char in words if char not in string.punctuation]
    stop.append(''.join(s))
stop.extend(['may','also','across','among','beside','however','yet','within'])

# stemmer = SnowballStemmer("english")
# def stemming(sentence):
#     stemSentence = ""
#     for word in sentence.split():
#         stem = stemmer.stem(word)
#         stemSentence += stem
#         stemSentence += " "
#     stemSentence = stemSentence.strip()
#     return stemSentence

def process_normal(text):
    return [word for word in text.split() if word.lower() not in stop]

def process_nocase(text):
    return [word.lower() for word in text.split() if word.lower() not in stop]

def process_nopunc(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stop]

def process_nocasepunc(text):
    nocasepunc = [char for char in text if char not in string.punctuation]
    nocasepunc = ''.join(nocasepunc)
    return [word.lower() for word in nocasepunc.split() if word.lower() not in stop]

# Pipeline

In [12]:
listAnalyzer = [process_normal, process_nocase,process_nopunc,process_nocasepunc]
nama = ['normal','noCase','noPunc','noCasePunc']
dPP = {}
dPP2 = {}

# buat analyzer
for i in range(0,4): 
#     buat n_gram
    for j in range(1,4):
        dPP['pp_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('classifier', MultinomialNB()),
        ])

        # With TFIDF
        dPP2['pp2_{}_ngram_{}'.format(nama[i],j)] = Pipeline([
            ('bow', CountVectorizer(analyzer=listAnalyzer[i], ngram_range=(1,j), lowercase=False)),
            ('tfidf', TfidfTransformer()),
            ('classifier', MultinomialNB()),
        ])

In [13]:
print('OwO')

OwO


# Predict

In [14]:
ppKey = list(dPP.keys())
pp2Key = list(dPP2.keys())

dPredict_train = {}
dPredict_test = {}
dPredictProba_train = {}
dPredictProba_test = {}

dPredict2_train = {}
dPredict2_test = {}
dPredict2Proba_train = {}
dPredict2Proba_test = {}

for item, item2 in zip(ppKey, pp2Key):
    for i in range(1,8):
#         Pipeline
        dPP[item].fit(X_train, dSplit['y{}_train'.format(i)])
        
        dPredict_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict(X_train)
        dPredictProba_train['predictions_train_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_train)
        
        dPredict_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict(X_test)
        dPredictProba_test['predictions_test_y{}_{}'.format(i,item)] = dPP[item].predict_proba(X_test)
        
#         Pipeline2
        dPP2[item2].fit(X_train, dSplit['y{}_train'.format(i)])
    
        dPredict2_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_train)
        dPredict2Proba_train['predictions2_train_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_train)
        
        dPredict2_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict(X_test)
        dPredict2Proba_test['predictions2_test_y{}_{}'.format(i,item2)] = dPP2[item2].predict_proba(X_test)

In [15]:
print('OwO')

OwO


# Report

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, matthews_corrcoef, f1_score, log_loss

In [17]:
def calc_train_error(X_train, y_train, predictTrain, predictProbaTrain):
    matt = matthews_corrcoef(y_train, predictTrain)
    f1 = f1_score(y_train, predictTrain, average = 'macro')
    report = classification_report(y_train, predictTrain)
    roc_auc = roc_auc_score(y_train, predictProbaTrain[:,1])
    accuracy = accuracy_score(y_train, predictTrain)
    confMatrix = confusion_matrix(y_train, predictTrain)
    logloss = log_loss(y_train, predictProbaTrain)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_validation_error(X_test, y_test, predictTest, predictProbaTest):
    matt = matthews_corrcoef(y_test, predictTest)
    f1 = f1_score(y_test, predictTest, average = 'macro')
    report = classification_report(y_test, predictTest)
    roc_auc = roc_auc_score(y_test, predictProbaTest[:,1])
    accuracy = accuracy_score(y_test, predictTest)
    confMatrix = confusion_matrix(y_test, predictTest)
    logloss = log_loss(y_test, predictProbaTest)
    return {
        'report':report,
        'matthew':matt,
        'f1':f1,
        'roc':roc_auc,
        'accuracy':accuracy,
        'confusion':confMatrix,
        'logloss':logloss
    }

def calc_metrics(X_train, y_train, X_test, y_test, predictTrain, predictProbaTrain, predictTest, predictProbaTest):
    train_error = calc_train_error(X_train, y_train, predictTrain, predictProbaTrain)
    validation_error = calc_validation_error(X_test, y_test, predictTest, predictProbaTest)
    return train_error, validation_error

## Toxic

In [21]:
dToxic_train = {}
dToxic_test = {}
dToxic2_train = {}
dToxic2_test = {}

for item in ppKey:
    dToxic_train['hasil_train_{}'.format(item)], dToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y1_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y1_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y1_{}'.format(item)])

for item2 in pp2Key:
    dToxic2_train['hasil_train_{}'.format(item2)], dToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y1_train'], X_test, dSplit['y1_test'], 
                                                                                                          dPredict2_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y1_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y1_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y1_{}'.format(item2)])

In [23]:
dToxic_train_key = list(dToxic_train.keys())
dToxic_test_key = list(dToxic_test.keys())
listItem = []

for train, test in zip(dToxic_train_key, dToxic_test_key):
    listItem.append(['nb_{}'.format(test),
                     dToxic_train[train]['accuracy'],
                     dToxic_test[test]['accuracy'],
                     dToxic_train[train]['roc'],
                     dToxic_test[test]['roc'],
                     dToxic_train[train]['f1'],
                     dToxic_test[test]['f1'],
                     dToxic_train[train]['matthew'],
                     dToxic_test[test]['matthew'],
                     dToxic_train[train]['logloss'],
                     dToxic_test[test]['logloss'],])


dfToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.951898,0.939902,0.918139,0.865366,0.846349,0.784868,0.698886,0.599035,0.426911,0.669210
1,nb_hasil_test_pp_normal_ngram_2,0.951898,0.939902,0.918139,0.865366,0.846349,0.784868,0.698886,0.599035,0.426911,0.669210
2,nb_hasil_test_pp_normal_ngram_3,0.951898,0.939902,0.918139,0.865366,0.846349,0.784868,0.698886,0.599035,0.426911,0.669210
3,nb_hasil_test_pp_noCase_ngram_1,0.955613,0.943412,0.922005,0.871070,0.856230,0.794398,0.720778,0.623761,0.359005,0.588860
4,nb_hasil_test_pp_noCase_ngram_2,0.955613,0.943412,0.922005,0.871070,0.856230,0.794398,0.720778,0.623761,0.359005,0.588860
5,nb_hasil_test_pp_noCase_ngram_3,0.955613,0.943412,0.922005,0.871070,0.856230,0.794398,0.720778,0.623761,0.359005,0.588860
6,nb_hasil_test_pp_noPunc_ngram_1,0.950814,0.943892,0.920406,0.885848,0.847951,0.811978,0.698634,0.638196,0.396559,0.534276
7,nb_hasil_test_pp_noPunc_ngram_2,0.950814,0.943892,0.920406,0.885848,0.847951,0.811978,0.698634,0.638196,0.396559,0.534276
8,nb_hasil_test_pp_noPunc_ngram_3,0.950814,0.943892,0.920406,0.885848,0.847951,0.811978,0.698634,0.638196,0.396559,0.534276
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.954064,0.948279,0.924392,0.893266,0.856380,0.825367,0.716595,0.667063,0.322577,0.442304


In [27]:
dToxic2_train_key = list(dToxic2_train.keys())
dToxic2_test_key = list(dToxic2_test.keys())
listItem = []

for train, test in zip(dToxic2_train_key, dToxic2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dToxic2_train[train]['accuracy'],
                     dToxic2_test[test]['accuracy'],
                     dToxic2_train[train]['roc'],
                     dToxic2_test[test]['roc'],
                     dToxic2_train[train]['f1'],
                     dToxic2_test[test]['f1'],
                     dToxic2_train[train]['matthew'],
                     dToxic2_test[test]['matthew'],
                     dToxic2_train[train]['logloss'],
                     dToxic2_test[test]['logloss'],])


dfToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.921387,0.917091,0.885284,0.858003,0.632937,0.601998,0.404170,0.356011,0.305950,0.374684
1,nb_hasil_test_pp2_normal_ngram_2,0.921387,0.917091,0.885284,0.858003,0.632937,0.601998,0.404170,0.356011,0.305950,0.374684
2,nb_hasil_test_pp2_normal_ngram_3,0.921387,0.917091,0.885284,0.858003,0.632937,0.601998,0.404170,0.356011,0.305950,0.374684
3,nb_hasil_test_pp2_noCase_ngram_1,0.917340,0.913895,0.882513,0.856362,0.598157,0.573145,0.352729,0.310479,0.316984,0.383680
4,nb_hasil_test_pp2_noCase_ngram_2,0.917340,0.913895,0.882513,0.856362,0.598157,0.573145,0.352729,0.310479,0.316984,0.383680
5,nb_hasil_test_pp2_noCase_ngram_3,0.917340,0.913895,0.882513,0.856362,0.598157,0.573145,0.352729,0.310479,0.316984,0.383680
6,nb_hasil_test_pp2_noPunc_ngram_1,0.926535,0.921771,0.892184,0.876231,0.673583,0.644436,0.462072,0.413889,0.276669,0.322398
7,nb_hasil_test_pp2_noPunc_ngram_2,0.926535,0.921771,0.892184,0.876231,0.673583,0.644436,0.462072,0.413889,0.276669,0.322398
8,nb_hasil_test_pp2_noPunc_ngram_3,0.926535,0.921771,0.892184,0.876231,0.673583,0.644436,0.462072,0.413889,0.276669,0.322398
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.922694,0.919076,0.889036,0.875096,0.641378,0.616785,0.420129,0.382350,0.283584,0.326161


In [ ]:
# print(confusion_matrix(dSplit['y1_test'], predictions))
# print('\n')
# print(classification_report(dSplit['y1_test'], predictions))

## Severe Toxic

In [28]:
dSToxic_train = {}
dSToxic_test = {}
dSToxic2_train = {}
dSToxic2_test = {}

for item in ppKey:
    dSToxic_train['hasil_train_{}'.format(item)], dSToxic_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y2_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y2_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y2_{}'.format(item)])

for item2 in pp2Key:
    dSToxic2_train['hasil_train_{}'.format(item2)], dSToxic2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y2_train'], X_test, dSplit['y2_test'], 
                                                                                                          dPredict2_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y2_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y2_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y2_{}'.format(item2)])

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being 

In [29]:
dSToxic_train_key = list(dSToxic_train.keys())
dSToxic_test_key = list(dSToxic_test.keys())
listItem = []

for train, test in zip(dSToxic_train_key, dSToxic_test_key):
    listItem.append(['nb_{}'.format(test),
                     dSToxic_train[train]['accuracy'],
                     dSToxic_test[test]['accuracy'],
                     dSToxic_train[train]['roc'],
                     dSToxic_test[test]['roc'],
                     dSToxic_train[train]['f1'],
                     dSToxic_test[test]['f1'],
                     dSToxic_train[train]['matthew'],
                     dSToxic_test[test]['matthew'],
                     dSToxic_train[train]['logloss'],
                     dSToxic_test[test]['logloss'],])


dfSToxicResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxicResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.985103,0.986401,0.919487,0.877371,0.693904,0.662719,0.396885,0.325613,0.147351,0.170952
1,nb_hasil_test_pp_normal_ngram_2,0.985103,0.986401,0.919487,0.877371,0.693904,0.662719,0.396885,0.325613,0.147351,0.170952
2,nb_hasil_test_pp_normal_ngram_3,0.985103,0.986401,0.919487,0.877371,0.693904,0.662719,0.396885,0.325613,0.147351,0.170952
3,nb_hasil_test_pp_noCase_ngram_1,0.989812,0.989326,0.920979,0.869955,0.729285,0.671832,0.458775,0.358833,0.097431,0.147400
4,nb_hasil_test_pp_noCase_ngram_2,0.989812,0.989326,0.920979,0.869955,0.729285,0.671832,0.458775,0.358833,0.097431,0.147400
5,nb_hasil_test_pp_noCase_ngram_3,0.989812,0.989326,0.920979,0.869955,0.729285,0.671832,0.458775,0.358833,0.097431,0.147400
6,nb_hasil_test_pp_noPunc_ngram_1,0.982686,0.984855,0.926575,0.896209,0.685873,0.679574,0.391098,0.362004,0.165142,0.176905
7,nb_hasil_test_pp_noPunc_ngram_2,0.982686,0.984855,0.926575,0.896209,0.685873,0.679574,0.391098,0.362004,0.165142,0.176905
8,nb_hasil_test_pp_noPunc_ngram_3,0.982686,0.984855,0.926575,0.896209,0.685873,0.679574,0.391098,0.362004,0.165142,0.176905
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.988236,0.988448,0.928730,0.894746,0.725987,0.698165,0.454314,0.397934,0.100452,0.129815


In [31]:
dSToxic2_train_key = list(dSToxic2_train.keys())
dSToxic2_test_key = list(dSToxic2_test.keys())
listItem = []

for train, test in zip(dSToxic2_train_key, dSToxic2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dSToxic2_train[train]['accuracy'],
                     dSToxic2_test[test]['accuracy'],
                     dSToxic2_train[train]['roc'],
                     dSToxic2_test[test]['roc'],
                     dSToxic2_train[train]['f1'],
                     dSToxic2_test[test]['f1'],
                     dSToxic2_train[train]['matthew'],
                     dSToxic2_test[test]['matthew'],
                     dSToxic2_train[train]['logloss'],
                     dSToxic2_test[test]['logloss'],])


dfSToxic2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfSToxic2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.990224,0.989493,0.889570,0.869954,0.499369,0.497359,0.029816,0.000000,0.064492,0.081846
1,nb_hasil_test_pp2_normal_ngram_2,0.990224,0.989493,0.889570,0.869954,0.499369,0.497359,0.029816,0.000000,0.064492,0.081846
2,nb_hasil_test_pp2_normal_ngram_3,0.990224,0.989493,0.889570,0.869954,0.499369,0.497359,0.029816,0.000000,0.064492,0.081846
3,nb_hasil_test_pp2_noCase_ngram_1,0.990215,0.989493,0.877326,0.854195,0.497542,0.497359,-0.000297,0.000000,0.071782,0.089553
4,nb_hasil_test_pp2_noCase_ngram_2,0.990215,0.989493,0.877326,0.854195,0.497542,0.497359,-0.000297,0.000000,0.071782,0.089553
5,nb_hasil_test_pp2_noCase_ngram_3,0.990215,0.989493,0.877326,0.854195,0.497542,0.497359,-0.000297,0.000000,0.071782,0.089553
6,nb_hasil_test_pp2_noPunc_ngram_1,0.990197,0.989514,0.893684,0.883946,0.500262,0.499349,0.029520,0.044353,0.062866,0.076957
7,nb_hasil_test_pp2_noPunc_ngram_2,0.990197,0.989514,0.893684,0.883946,0.500262,0.499349,0.029520,0.044353,0.062866,0.076957
8,nb_hasil_test_pp2_noPunc_ngram_3,0.990197,0.989514,0.893684,0.883946,0.500262,0.499349,0.029520,0.044353,0.062866,0.076957
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.990197,0.989472,0.881689,0.873722,0.497537,0.497354,-0.000515,-0.000471,0.070197,0.083772


## Obscene

In [42]:
dObscene_train = {}
dObscene_test = {}
dObscene2_train = {}
dObscene2_test = {}

for item in ppKey:
    dObscene_train['hasil_train_{}'.format(item)], dObscene_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y3_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y3_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y3_{}'.format(item)])

for item2 in pp2Key:
    dObscene2_train['hasil_train_{}'.format(item2)], dObscene2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y3_train'], X_test, dSplit['y3_test'], 
                                                                                                          dPredict2_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y3_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y3_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y3_{}'.format(item2)])

In [43]:
dObscene_train_key = list(dObscene_train.keys())
dObscene_test_key = list(dObscene_test.keys())
listItem = []

for train, test in zip(dObscene_train_key, dObscene_test_key):
    listItem.append(['nb_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfObsceneResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObsceneResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.965309,0.961105,0.908342,0.866832,0.816778,0.767424,0.634710,0.552068,0.352572,0.470132
1,nb_hasil_test_pp_normal_ngram_2,0.965309,0.961105,0.908342,0.866832,0.816778,0.767424,0.634710,0.552068,0.352572,0.470132
2,nb_hasil_test_pp_normal_ngram_3,0.965309,0.961105,0.908342,0.866832,0.816778,0.767424,0.634710,0.552068,0.352572,0.470132
3,nb_hasil_test_pp_noCase_ngram_1,0.970823,0.965429,0.911857,0.872463,0.836993,0.785896,0.679640,0.598398,0.287314,0.414655
4,nb_hasil_test_pp_noCase_ngram_2,0.970823,0.965429,0.911857,0.872463,0.836993,0.785896,0.679640,0.598398,0.287314,0.414655
5,nb_hasil_test_pp_noCase_ngram_3,0.970823,0.965429,0.911857,0.872463,0.836993,0.785896,0.679640,0.598398,0.287314,0.414655
6,nb_hasil_test_pp_noPunc_ngram_1,0.964037,0.962588,0.915026,0.887065,0.819675,0.796132,0.639351,0.596944,0.335241,0.404055
7,nb_hasil_test_pp_noPunc_ngram_2,0.964037,0.962588,0.915026,0.887065,0.819675,0.796132,0.639351,0.596944,0.335241,0.404055
8,nb_hasil_test_pp_noPunc_ngram_3,0.964037,0.962588,0.915026,0.887065,0.819675,0.796132,0.639351,0.596944,0.335241,0.404055
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.970232,0.969105,0.920280,0.894862,0.842971,0.824557,0.687120,0.659487,0.253209,0.324471


In [44]:
dObscene2_train_key = list(dObscene2_train.keys())
dObscene2_test_key = list(dObscene2_test.keys())
listItem = []

for train, test in zip(dObscene2_train_key, dObscene2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfObscene2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfObscene2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.952730,0.950180,0.882045,0.859319,0.580799,0.556513,0.310407,0.263924,0.212178,0.261378
1,nb_hasil_test_pp2_normal_ngram_2,0.952730,0.950180,0.882045,0.859319,0.580799,0.556513,0.310407,0.263924,0.212178,0.261378
2,nb_hasil_test_pp2_normal_ngram_3,0.952730,0.950180,0.882045,0.859319,0.580799,0.556513,0.310407,0.263924,0.212178,0.261378
3,nb_hasil_test_pp2_noCase_ngram_1,0.951074,0.949093,0.879545,0.859339,0.553201,0.538099,0.257488,0.224587,0.222400,0.269822
4,nb_hasil_test_pp2_noCase_ngram_2,0.951074,0.949093,0.879545,0.859339,0.553201,0.538099,0.257488,0.224587,0.222400,0.269822
5,nb_hasil_test_pp2_noCase_ngram_3,0.951074,0.949093,0.879545,0.859339,0.553201,0.538099,0.257488,0.224587,0.222400,0.269822
6,nb_hasil_test_pp2_noPunc_ngram_1,0.955837,0.952457,0.890051,0.875959,0.630157,0.593427,0.391017,0.331719,0.193266,0.228740
7,nb_hasil_test_pp2_noPunc_ngram_2,0.955837,0.952457,0.890051,0.875959,0.630157,0.593427,0.391017,0.331719,0.193266,0.228740
8,nb_hasil_test_pp2_noPunc_ngram_3,0.955837,0.952457,0.890051,0.875959,0.630157,0.593427,0.391017,0.331719,0.193266,0.228740
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.953554,0.951120,0.888149,0.876910,0.594289,0.571336,0.333626,0.293894,0.200489,0.233979


## Threat

In [45]:
dThreat_train = {}
dThreat_test = {}
dThreat2_train = {}
dThreat2_test = {}

for item in ppKey:
    dThreat_train['hasil_train_{}'.format(item)], dThreat_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y4_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y4_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y4_{}'.format(item)])

for item2 in pp2Key:
    dThreat2_train['hasil_train_{}'.format(item2)], dThreat2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y4_train'], X_test, dSplit['y4_test'], 
                                                                                                          dPredict2_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y4_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y4_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y4_{}'.format(item2)])

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being 

In [46]:
dThreat_train_key = list(dThreat_train.keys())
dThreat_test_key = list(dThreat_test.keys())
listItem = []

for train, test in zip(dThreat_train_key, dThreat_test_key):
    listItem.append(['nb_{}'.format(test),
                     dObscene_train[train]['accuracy'],
                     dObscene_test[test]['accuracy'],
                     dObscene_train[train]['roc'],
                     dObscene_test[test]['roc'],
                     dObscene_train[train]['f1'],
                     dObscene_test[test]['f1'],
                     dObscene_train[train]['matthew'],
                     dObscene_test[test]['matthew'],
                     dObscene_train[train]['logloss'],
                     dObscene_test[test]['logloss'],])


dfThreatResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreatResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.965309,0.961105,0.908342,0.866832,0.816778,0.767424,0.634710,0.552068,0.352572,0.470132
1,nb_hasil_test_pp_normal_ngram_2,0.965309,0.961105,0.908342,0.866832,0.816778,0.767424,0.634710,0.552068,0.352572,0.470132
2,nb_hasil_test_pp_normal_ngram_3,0.965309,0.961105,0.908342,0.866832,0.816778,0.767424,0.634710,0.552068,0.352572,0.470132
3,nb_hasil_test_pp_noCase_ngram_1,0.970823,0.965429,0.911857,0.872463,0.836993,0.785896,0.679640,0.598398,0.287314,0.414655
4,nb_hasil_test_pp_noCase_ngram_2,0.970823,0.965429,0.911857,0.872463,0.836993,0.785896,0.679640,0.598398,0.287314,0.414655
5,nb_hasil_test_pp_noCase_ngram_3,0.970823,0.965429,0.911857,0.872463,0.836993,0.785896,0.679640,0.598398,0.287314,0.414655
6,nb_hasil_test_pp_noPunc_ngram_1,0.964037,0.962588,0.915026,0.887065,0.819675,0.796132,0.639351,0.596944,0.335241,0.404055
7,nb_hasil_test_pp_noPunc_ngram_2,0.964037,0.962588,0.915026,0.887065,0.819675,0.796132,0.639351,0.596944,0.335241,0.404055
8,nb_hasil_test_pp_noPunc_ngram_3,0.964037,0.962588,0.915026,0.887065,0.819675,0.796132,0.639351,0.596944,0.335241,0.404055
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.970232,0.969105,0.920280,0.894862,0.842971,0.824557,0.687120,0.659487,0.253209,0.324471


In [47]:
dThreat2_train_key = list(dThreat2_train.keys())
dThreat2_test_key = list(dThreat2_test.keys())
listItem = []

for train, test in zip(dThreat2_train_key, dThreat2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dObscene2_train[train]['accuracy'],
                     dObscene2_test[test]['accuracy'],
                     dObscene2_train[train]['roc'],
                     dObscene2_test[test]['roc'],
                     dObscene2_train[train]['f1'],
                     dObscene2_test[test]['f1'],
                     dObscene2_train[train]['matthew'],
                     dObscene2_test[test]['matthew'],
                     dObscene2_train[train]['logloss'],
                     dObscene2_test[test]['logloss'],])


dfThreat2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfThreat2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.952730,0.950180,0.882045,0.859319,0.580799,0.556513,0.310407,0.263924,0.212178,0.261378
1,nb_hasil_test_pp2_normal_ngram_2,0.952730,0.950180,0.882045,0.859319,0.580799,0.556513,0.310407,0.263924,0.212178,0.261378
2,nb_hasil_test_pp2_normal_ngram_3,0.952730,0.950180,0.882045,0.859319,0.580799,0.556513,0.310407,0.263924,0.212178,0.261378
3,nb_hasil_test_pp2_noCase_ngram_1,0.951074,0.949093,0.879545,0.859339,0.553201,0.538099,0.257488,0.224587,0.222400,0.269822
4,nb_hasil_test_pp2_noCase_ngram_2,0.951074,0.949093,0.879545,0.859339,0.553201,0.538099,0.257488,0.224587,0.222400,0.269822
5,nb_hasil_test_pp2_noCase_ngram_3,0.951074,0.949093,0.879545,0.859339,0.553201,0.538099,0.257488,0.224587,0.222400,0.269822
6,nb_hasil_test_pp2_noPunc_ngram_1,0.955837,0.952457,0.890051,0.875959,0.630157,0.593427,0.391017,0.331719,0.193266,0.228740
7,nb_hasil_test_pp2_noPunc_ngram_2,0.955837,0.952457,0.890051,0.875959,0.630157,0.593427,0.391017,0.331719,0.193266,0.228740
8,nb_hasil_test_pp2_noPunc_ngram_3,0.955837,0.952457,0.890051,0.875959,0.630157,0.593427,0.391017,0.331719,0.193266,0.228740
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.953554,0.951120,0.888149,0.876910,0.594289,0.571336,0.333626,0.293894,0.200489,0.233979


## Insult

In [48]:
dInsult_train = {}
dInsult_test = {}
dInsult2_train = {}
dInsult2_test = {}

for item in ppKey:
    dInsult_train['hasil_train_{}'.format(item)], dInsult_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y5_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y5_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y5_{}'.format(item)])

for item2 in pp2Key:
    dInsult2_train['hasil_train_{}'.format(item2)], dInsult2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y5_train'], X_test, dSplit['y5_test'], 
                                                                                                          dPredict2_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y5_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y5_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y5_{}'.format(item2)])

In [49]:
dInsult_train_key = list(dInsult_train.keys())
dInsult_test_key = list(dInsult_test.keys())
listItem = []

for train, test in zip(dInsult_train_key, dInsult_test_key):
    listItem.append(['nb_{}'.format(test),
                     dInsult_train[train]['accuracy'],
                     dInsult_test[test]['accuracy'],
                     dInsult_train[train]['roc'],
                     dInsult_test[test]['roc'],
                     dInsult_train[train]['f1'],
                     dInsult_test[test]['f1'],
                     dInsult_train[train]['matthew'],
                     dInsult_test[test]['matthew'],
                     dInsult_train[train]['logloss'],
                     dInsult_test[test]['logloss'],])


dfInsultResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsultResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.962596,0.958723,0.902498,0.855593,0.789018,0.729254,0.579286,0.475850,0.372635,0.505798
1,nb_hasil_test_pp_normal_ngram_2,0.962596,0.958723,0.902498,0.855593,0.789018,0.729254,0.579286,0.475850,0.372635,0.505798
2,nb_hasil_test_pp_normal_ngram_3,0.962596,0.958723,0.902498,0.855593,0.789018,0.729254,0.579286,0.475850,0.372635,0.505798
3,nb_hasil_test_pp_noCase_ngram_1,0.967753,0.963089,0.905225,0.860663,0.805787,0.745199,0.618185,0.520949,0.308122,0.443069
4,nb_hasil_test_pp_noCase_ngram_2,0.967753,0.963089,0.905225,0.860663,0.805787,0.745199,0.618185,0.520949,0.308122,0.443069
5,nb_hasil_test_pp_noCase_ngram_3,0.967753,0.963089,0.905225,0.860663,0.805787,0.745199,0.618185,0.520949,0.308122,0.443069
6,nb_hasil_test_pp_noPunc_ngram_1,0.961110,0.959329,0.908576,0.876214,0.792534,0.757855,0.585068,0.520694,0.358021,0.440286
7,nb_hasil_test_pp_noPunc_ngram_2,0.961110,0.959329,0.908576,0.876214,0.792534,0.757855,0.585068,0.520694,0.358021,0.440286
8,nb_hasil_test_pp_noPunc_ngram_3,0.961110,0.959329,0.908576,0.876214,0.792534,0.757855,0.585068,0.520694,0.358021,0.440286
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.966839,0.965679,0.912866,0.883744,0.813535,0.786131,0.628262,0.583515,0.276083,0.352919


In [50]:
dInsult2_train_key = list(dInsult2_train.keys())
dInsult2_test_key = list(dInsult2_test.keys())
listItem = []

for train, test in zip(dInsult2_train_key, dInsult2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dInsult2_train[train]['accuracy'],
                     dInsult2_test[test]['accuracy'],
                     dInsult2_train[train]['roc'],
                     dInsult2_test[test]['roc'],
                     dInsult2_train[train]['f1'],
                     dInsult2_test[test]['f1'],
                     dInsult2_train[train]['matthew'],
                     dInsult2_test[test]['matthew'],
                     dInsult2_train[train]['logloss'],
                     dInsult2_test[test]['logloss'],])


dfInsult2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfInsult2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.952945,0.951537,0.875084,0.849800,0.532149,0.509416,0.208259,0.141499,0.216749,0.261584
1,nb_hasil_test_pp2_normal_ngram_2,0.952945,0.951537,0.875084,0.849800,0.532149,0.509416,0.208259,0.141499,0.216749,0.261584
2,nb_hasil_test_pp2_normal_ngram_3,0.952945,0.951537,0.875084,0.849800,0.532149,0.509416,0.208259,0.141499,0.216749,0.261584
3,nb_hasil_test_pp2_noCase_ngram_1,0.951477,0.951078,0.871706,0.849791,0.504146,0.500350,0.122452,0.105970,0.228451,0.269919
4,nb_hasil_test_pp2_noCase_ngram_2,0.951477,0.951078,0.871706,0.849791,0.504146,0.500350,0.122452,0.105970,0.228451,0.269919
5,nb_hasil_test_pp2_noCase_ngram_3,0.951477,0.951078,0.871706,0.849791,0.504146,0.500350,0.122452,0.105970,0.228451,0.269919
6,nb_hasil_test_pp2_noPunc_ngram_1,0.954655,0.952895,0.882263,0.865914,0.567319,0.538882,0.276534,0.215049,0.200580,0.233618
7,nb_hasil_test_pp2_noPunc_ngram_2,0.954655,0.952895,0.882263,0.865914,0.567319,0.538882,0.276534,0.215049,0.200580,0.233618
8,nb_hasil_test_pp2_noPunc_ngram_3,0.954655,0.952895,0.882263,0.865914,0.567319,0.538882,0.276534,0.215049,0.200580,0.233618
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.952658,0.951746,0.879414,0.866520,0.528345,0.513872,0.194400,0.155031,0.209573,0.239064


## Identity Hate

In [51]:
dIdHate_train = {}
dIdHate_test = {}
dIdHate2_train = {}
dIdHate2_test = {}

for item in ppKey:
    dIdHate_train['hasil_train_{}'.format(item)], dIdHate_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y6_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y6_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y6_{}'.format(item)])

for item2 in pp2Key:
    dIdHate2_train['hasil_train_{}'.format(item2)], dIdHate2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y6_train'], X_test, dSplit['y6_test'], 
                                                                                                          dPredict2_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y6_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y6_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y6_{}'.format(item2)])

C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:543: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\Users\Asus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being 

In [52]:
dIdHate_train_key = list(dIdHate_train.keys())
dIdHate_test_key = list(dIdHate_test.keys())
listItem = []

for train, test in zip(dIdHate_train_key, dIdHate_test_key):
    listItem.append(['nb_{}'.format(test),
                     dIdHate_train[train]['accuracy'],
                     dIdHate_test[test]['accuracy'],
                     dIdHate_train[train]['roc'],
                     dIdHate_test[test]['roc'],
                     dIdHate_train[train]['f1'],
                     dIdHate_test[test]['f1'],
                     dIdHate_train[train]['matthew'],
                     dIdHate_test[test]['matthew'],
                     dIdHate_train[train]['logloss'],
                     dIdHate_test[test]['logloss'],])


dfIdHateResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHateResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.987019,0.990287,0.845152,0.823178,0.621468,0.588946,0.243053,0.199516,0.150935,0.148541
1,nb_hasil_test_pp_normal_ngram_2,0.987019,0.990287,0.845152,0.823178,0.621468,0.588946,0.243053,0.199516,0.150935,0.148541
2,nb_hasil_test_pp_normal_ngram_3,0.987019,0.990287,0.845152,0.823178,0.621468,0.588946,0.243053,0.199516,0.150935,0.148541
3,nb_hasil_test_pp_noCase_ngram_1,0.990286,0.991415,0.837658,0.812998,0.614164,0.574131,0.251043,0.208392,0.131638,0.151029
4,nb_hasil_test_pp_noCase_ngram_2,0.990286,0.991415,0.837658,0.812998,0.614164,0.574131,0.251043,0.208392,0.131638,0.151029
5,nb_hasil_test_pp_noCase_ngram_3,0.990286,0.991415,0.837658,0.812998,0.614164,0.574131,0.251043,0.208392,0.131638,0.151029
6,nb_hasil_test_pp_noPunc_ngram_1,0.985040,0.988219,0.851130,0.841497,0.616967,0.603178,0.235129,0.208728,0.160919,0.148097
7,nb_hasil_test_pp_noPunc_ngram_2,0.985040,0.988219,0.851130,0.841497,0.616967,0.603178,0.235129,0.208728,0.160919,0.148097
8,nb_hasil_test_pp_noPunc_ngram_3,0.985040,0.988219,0.851130,0.841497,0.616967,0.603178,0.235129,0.208728,0.160919,0.148097
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.989436,0.990955,0.846983,0.832678,0.629509,0.603372,0.267293,0.239343,0.130446,0.135745


In [53]:
dIdHate2_train_key = list(dIdHate2_train.keys())
dIdHate2_test_key = list(dIdHate2_test.keys())
listItem = []

for train, test in zip(dIdHate2_train_key, dIdHate2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dIdHate2_train[train]['accuracy'],
                     dIdHate2_test[test]['accuracy'],
                     dIdHate2_train[train]['roc'],
                     dIdHate2_test[test]['roc'],
                     dIdHate2_train[train]['f1'],
                     dIdHate2_test[test]['f1'],
                     dIdHate2_train[train]['matthew'],
                     dIdHate2_test[test]['matthew'],
                     dIdHate2_train[train]['logloss'],
                     dIdHate2_test[test]['logloss'],])


dfIdHate2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfIdHate2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.991065,0.991435,0.828696,0.828722,0.497756,0.49785,-0.000491,0.0,0.070718,0.074260
1,nb_hasil_test_pp2_normal_ngram_2,0.991065,0.991435,0.828696,0.828722,0.497756,0.49785,-0.000491,0.0,0.070718,0.074260
2,nb_hasil_test_pp2_normal_ngram_3,0.991065,0.991435,0.828696,0.828722,0.497756,0.49785,-0.000491,0.0,0.070718,0.074260
3,nb_hasil_test_pp2_noCase_ngram_1,0.991083,0.991435,0.817748,0.814807,0.497761,0.49785,-0.000284,0.0,0.075750,0.079268
4,nb_hasil_test_pp2_noCase_ngram_2,0.991083,0.991435,0.817748,0.814807,0.497761,0.49785,-0.000284,0.0,0.075750,0.079268
5,nb_hasil_test_pp2_noCase_ngram_3,0.991083,0.991435,0.817748,0.814807,0.497761,0.49785,-0.000284,0.0,0.075750,0.079268
6,nb_hasil_test_pp2_noPunc_ngram_1,0.991038,0.991435,0.831442,0.840652,0.497750,0.49785,-0.000695,0.0,0.069702,0.070886
7,nb_hasil_test_pp2_noPunc_ngram_2,0.991038,0.991435,0.831442,0.840652,0.497750,0.49785,-0.000695,0.0,0.069702,0.070886
8,nb_hasil_test_pp2_noPunc_ngram_3,0.991038,0.991435,0.831442,0.840652,0.497750,0.49785,-0.000695,0.0,0.069702,0.070886
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.991065,0.991435,0.821046,0.827072,0.497756,0.49785,-0.000491,0.0,0.074530,0.075820


## Clean

In [55]:
dClean_train = {}
dClean_test = {}
dClean2_train = {}
dClean2_test = {}

for item in ppKey:
    dClean_train['hasil_train_{}'.format(item)], dClean_test['hasil_test_{}'.format(item)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredictProba_train['predictions_train_y7_{}'.format(item)],
                                                                                                          dPredict_test['predictions_test_y7_{}'.format(item)],
                                                                                                          dPredictProba_test['predictions_test_y7_{}'.format(item)])

for item2 in pp2Key:
    dClean2_train['hasil_train_{}'.format(item2)], dClean2_test['hasil_test_{}'.format(item2)]= calc_metrics(X_train, dSplit['y7_train'], X_test, dSplit['y7_test'], 
                                                                                                          dPredict2_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_train['predictions2_train_y7_{}'.format(item2)],
                                                                                                          dPredict2_test['predictions2_test_y7_{}'.format(item2)],
                                                                                                          dPredict2Proba_test['predictions2_test_y7_{}'.format(item2)])

In [56]:
dClean_train_key = list(dClean_train.keys())
dClean_test_key = list(dClean_test.keys())
listItem = []

for train, test in zip(dClean_train_key, dClean_test_key):
    listItem.append(['nb_{}'.format(test),
                     dClean_train[train]['accuracy'],
                     dClean_test[test]['accuracy'],
                     dClean_train[train]['roc'],
                     dClean_test[test]['roc'],
                     dClean_train[train]['f1'],
                     dClean_test[test]['f1'],
                     dClean_train[train]['matthew'],
                     dClean_test[test]['matthew'],
                     dClean_train[train]['logloss'],
                     dClean_test[test]['logloss'],])


dfCleanResult = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfCleanResult

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp_normal_ngram_1,0.951539,0.937061,0.918958,0.860332,0.853770,0.787366,0.713528,0.603130,0.422280,0.694240
1,nb_hasil_test_pp_normal_ngram_2,0.951539,0.937061,0.918958,0.860332,0.853770,0.787366,0.713528,0.603130,0.422280,0.694240
2,nb_hasil_test_pp_normal_ngram_3,0.951539,0.937061,0.918958,0.860332,0.853770,0.787366,0.713528,0.603130,0.422280,0.694240
3,nb_hasil_test_pp_noCase_ngram_1,0.955085,0.941490,0.923482,0.867775,0.862915,0.799666,0.733550,0.633049,0.354129,0.606094
4,nb_hasil_test_pp_noCase_ngram_2,0.955085,0.941490,0.923482,0.867775,0.862915,0.799666,0.733550,0.633049,0.354129,0.606094
5,nb_hasil_test_pp_noCase_ngram_3,0.955085,0.941490,0.923482,0.867775,0.862915,0.799666,0.733550,0.633049,0.354129,0.606094
6,nb_hasil_test_pp_noPunc_ngram_1,0.950385,0.942012,0.921447,0.882493,0.854975,0.816297,0.712571,0.646445,0.391685,0.550449
7,nb_hasil_test_pp_noPunc_ngram_2,0.950385,0.942012,0.921447,0.882493,0.854975,0.816297,0.712571,0.646445,0.391685,0.550449
8,nb_hasil_test_pp_noPunc_ngram_3,0.950385,0.942012,0.921447,0.882493,0.854975,0.816297,0.712571,0.646445,0.391685,0.550449
9,nb_hasil_test_pp_noCasePunc_ngram_1,0.953339,0.946420,0.926575,0.891142,0.862560,0.829155,0.728476,0.673976,0.315859,0.452114


In [57]:
dClean2_train_key = list(dClean2_train.keys())
dClean2_test_key = list(dClean2_test.keys())
listItem = []

for train, test in zip(dClean2_train_key, dClean2_test_key):
    listItem.append(['nb_{}'.format(test),
                     dClean2_train[train]['accuracy'],
                     dClean2_test[test]['accuracy'],
                     dClean2_train[train]['roc'],
                     dClean2_test[test]['roc'],
                     dClean2_train[train]['f1'],
                     dClean2_test[test]['f1'],
                     dClean2_train[train]['matthew'],
                     dClean2_test[test]['matthew'],
                     dClean2_train[train]['logloss'],
                     dClean2_test[test]['logloss'],])


dfClean2Result = pd.DataFrame(columns=['Model','Train Accuracy', 'Test Accuracy', 'Train ROC AUC', 'Test ROC AUC', 'Train F1 Score','Test F1 Score', 
                                      'Train Matthews Corr Coef', 'Test Matthew Corr Coef','Train Log Loss','Test Log Loss'], data=listItem)
dfClean2Result

,Model,Train Accuracy,Test Accuracy,Train ROC AUC,Test ROC AUC,Train F1 Score,Test F1 Score,Train Matthews Corr Coef,Test Matthew Corr Coef,Train Log Loss,Test Log Loss
0,nb_hasil_test_pp2_normal_ngram_1,0.917806,0.913248,0.884030,0.852359,0.640706,0.609753,0.417022,0.368362,0.315266,0.389430
1,nb_hasil_test_pp2_normal_ngram_2,0.917806,0.913248,0.884030,0.852359,0.640706,0.609753,0.417022,0.368362,0.315266,0.389430
2,nb_hasil_test_pp2_normal_ngram_3,0.917806,0.913248,0.884030,0.852359,0.640706,0.609753,0.417022,0.368362,0.315266,0.389430
3,nb_hasil_test_pp2_noCase_ngram_1,0.913938,0.910490,0.881795,0.851786,0.609761,0.585510,0.372860,0.333658,0.324774,0.395836
4,nb_hasil_test_pp2_noCase_ngram_2,0.913938,0.910490,0.881795,0.851786,0.609761,0.585510,0.372860,0.333658,0.324774,0.395836
5,nb_hasil_test_pp2_noCase_ngram_3,0.913938,0.910490,0.881795,0.851786,0.609761,0.585510,0.372860,0.333658,0.324774,0.395836
6,nb_hasil_test_pp2_noPunc_ngram_1,0.923876,0.918825,0.892058,0.871809,0.684272,0.654731,0.479195,0.431157,0.282728,0.332726
7,nb_hasil_test_pp2_noPunc_ngram_2,0.923876,0.918825,0.892058,0.871809,0.684272,0.654731,0.479195,0.431157,0.282728,0.332726
8,nb_hasil_test_pp2_noPunc_ngram_3,0.923876,0.918825,0.892058,0.871809,0.684272,0.654731,0.479195,0.431157,0.282728,0.332726
9,nb_hasil_test_pp2_noCasePunc_ngram_1,0.920510,0.916611,0.889527,0.871714,0.658019,0.632485,0.446395,0.408486,0.287972,0.333645


In [58]:
import pickle

In [64]:
dfToxicResult.to_csv('nb_toxic.csv',index=False)
dfToxic2Result.to_csv('nb_toxic2.csv',index=False)

dfSToxicResult.to_csv('nb_stoxic.csv',index=False)
dfSToxic2Result.to_csv('nb_stoxic2.csv',index=False)

dfObsceneResult.to_csv('nb_obscene.csv',index=False)
dfObscene2Result.to_csv('nb_obscene2.csv',index=False)

dfThreatResult.to_csv('nb_threat.csv',index=False)
dfThreat2Result.to_csv('nb_threat2.csv',index=False)

dfInsultResult.to_csv('nb_insult.csv',index=False)
dfInsult2Result.to_csv('nb_insult2.csv',index=False)

dfIdHateResult.to_csv('nb_idHate.csv',index=False)
dfIdHate2Result.to_csv('nb_idHate2.csv',index=False)

dfCleanResult.to_csv('nb_clean.csv',index=False)
dfClean2Result.to_csv('nb_clean2.csv',index=False)

In [63]:
model = '_nb'

pickle.dump(dToxic_train, open('dToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic_test, open('dToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_train, open('dToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dToxic2_test, open('dToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dSToxic_train, open('dSToxic_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic_test, open('dSToxic_test{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_train, open('dSToxic2_train{}.sav'.format(model), 'wb'))
pickle.dump(dSToxic2_test, open('dSToxic2_test{}.sav'.format(model), 'wb'))

pickle.dump(dObscene_train, open('dObscene_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene_test, open('dObscene_test{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_train, open('dObscene2_train{}.sav'.format(model), 'wb'))
pickle.dump(dObscene2_test, open('dObscene2_test{}.sav'.format(model), 'wb'))

pickle.dump(dThreat_train, open('dThreat_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat_test, open('dThreat_test{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_train, open('dThreat2_train{}.sav'.format(model), 'wb'))
pickle.dump(dThreat2_test, open('dThreat2_test{}.sav'.format(model), 'wb'))

pickle.dump(dInsult_train, open('dInsult_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult_test, open('dInsult_test{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_train, open('dInsult2_train{}.sav'.format(model), 'wb'))
pickle.dump(dInsult2_test, open('dInsult2_test{}.sav'.format(model), 'wb'))

pickle.dump(dIdHate_train, open('dIdHate_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate_test, open('dIdHate_test{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_train, open('dIdHate2_train{}.sav'.format(model), 'wb'))
pickle.dump(dIdHate2_test, open('dIdHate2_test{}.sav'.format(model), 'wb'))

pickle.dump(dClean_train, open('dClean_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean_test, open('dClean_test{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_train, open('dClean2_train{}.sav'.format(model), 'wb'))
pickle.dump(dClean2_test, open('dClean2_test{}.sav'.format(model), 'wb'))

In [65]:
pickle.dump(dPredict_train, open('dPredict_train.sav', 'wb'))
pickle.dump(dPredict_test, open('dPredict_test.sav', 'wb'))
pickle.dump(dPredictProba_train, open('dPredictProba_train.sav', 'wb'))
pickle.dump(dPredictProba_test, open('dPredictProba_test.sav', 'wb'))

pickle.dump(dPredict2_train, open('dPredict2_train.sav', 'wb'))
pickle.dump(dPredict2_test, open('dPredict2_test.sav', 'wb'))
pickle.dump(dPredict2Proba_train, open('dPredict2Proba_train.sav', 'wb'))
pickle.dump(dPredict2Proba_test, open('dPredict2Proba_test.sav', 'wb'))

In [67]:
X_test

64748     "  The point I think was very simply about bia...
10958     Please refrain from removing content from Wiki...
114719    See Tenth Amendment to the United States Const...
95438     "  Note that the warning was due to the fact t...
63391     See also User_talk:Webapp where I was blocked ...
22070     @ Samaleks - I have seen your allegation of SO...
121506    That official BBC link refers to it as series ...
125814    "   I am a WP:SPI clerk, so it is my job to wo...
101745    Soccer is only required to be used once per ar...
65758                   REDIRECT Talk:Donna Moss (designer)
11628     "   Your edits to Rabbit   Please stop replaci...
154068    Lying, scheming, vindictive Weasel   Burn in h...
25959     "  You know, don't be so sure of ""obviously f...
143816    " (outdent) Please don't feed the trolls by re...
112905    " which is something you are supposed to do as...
48585     "  Liberals  Btw Dave, ""Liberal"" means somth...
63697     The Santana   Hi, I'm starting